<a href="https://colab.research.google.com/github/AravindReddyGuda/New-folder/blob/master/SakiProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Installing required packages
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 17.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 23.8 MB/s eta 0:00:0

In [2]:
import torch
import datasets
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np


# Load the dataset
df = datasets.load_dataset("medical_questions_pairs")
df = pd.DataFrame(df["train"])

train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

train = datasets.Dataset.from_pandas(train_data)
test = datasets.Dataset.from_pandas(test_data)

MODEL = "deepset/roberta-base-squad2" 
#bert-base-uncased

# Define the tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)

# Tokenize the datasets
def tokenize_function(examples):
    return tokenizer(
                  examples['question_1'],
                  examples['question_2'],
                  add_special_tokens=True,      
                  max_length=125,                  
                  pad_to_max_length=True,         
                  return_attention_mask=True ,
                  truncation = True
              )

train_tokenised = train.map(tokenize_function, batched=True)
test_tokenised = test.map(tokenize_function, batched=True)


Generating train split:   0%|          | 0/3048 [00:00<?, ? examples/s]

Dataset medical_questions_pairs downloaded and prepared to /root/.cache/huggingface/datasets/medical_questions_pairs/default/0.0.0/db30a35b934dceb7abed5ef6b73a432bb59682d00e26f9a1acd960635333bc80. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/2438 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Map:   0%|          | 0/610 [00:00<?, ? examples/s]

In [3]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_dir='./logs',
    logging_steps=20,
    optim='adamw_torch',
    lr_scheduler_type='cosine_with_restarts',
    warmup_steps=50,
    weight_decay=0.01,
)

def compute_metrics(eval_pred):
    metric = datasets.load_metric('accuracy')
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenised,
    compute_metrics = compute_metrics
)

# Train the model
trainer.train()

eval_result = trainer.evaluate(test_tokenised)

eval_result

Some weights of the model checkpoint at deepset/roberta-base-squad2 were not used when initializing RobertaForSequenceClassification: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to

Step,Training Loss
20,0.693200
40,0.524700
60,0.531600
80,0.517900
100,0.411800
120,0.354700
140,0.369800
160,0.299000
180,0.221100
200,0.258800


<ipython-input-3-25977a499f19>:18: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('accuracy')


{'eval_loss': 0.5270459651947021,
 'eval_accuracy': 0.8754098360655738,
 'eval_runtime': 5.1434,
 'eval_samples_per_second': 118.599,
 'eval_steps_per_second': 3.889,
 'epoch': 5.0}

In [4]:
trainer.save_model('./')
tokenizer.save_pretrained('./')

('./tokenizer_config.json',
 './special_tokens_map.json',
 './vocab.json',
 './merges.txt',
 './added_tokens.json',
 './tokenizer.json')

In [5]:
from transformers import TextClassificationPipeline  #the textclassification pipeline is imported from hugging face transformers
 
model = AutoModelForSequenceClassification.from_pretrained('./')
tokenizer = AutoTokenizer.from_pretrained('./')
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer ) # We use our saved model and tokenizer

In [6]:
text = 'Enter two questions here seperated by a comma'
pipe(text)

[{'label': 'LABEL_0', 'score': 0.9921145439147949}]